# Dekoratoriai (funkcija funkcijoje)

Dekoratoriai priima funkcijas, prie tų funkcijų prideda papildomą funkcionalumą, ir grąžina rezultatą. Dekoratorius yra šiek tiek sudėtinga perprasti, ir galbūt praktikoje patiems niekada neprireiks rašyti savo dekoratoriaus. Tačiau jų veikimo principą suprasti reikia todėl, kad su jais nuolat susidursime įvairiuose Python framework'uose. Tam, kad lengviau suprasti dekoratorius, pravartu susipažinti su higher order functions (aukštesnio rango funkcijos?)

In [1]:
# Turime funkciją, kuri grąžina tekstą:
def return_text(some_text):
    return some_text

In [2]:
# Taip pat naudojame f-ją, kuri į parametrus priima kažkokį tekstą ir šalia jo funkciją:
def upper_text(text, func):
    if type(text) != str:
        raise ValueError("Argument type must be a string.")
    some_text = func(text)
    return some_text.upper()
# f-ja patikrina, ar gautas parametras yra string tipo ir grąžina rezultatą visomis didžiosiomis raidėmis.

In [5]:
# Tai yra vadinamoji higher order function, jai kaip antras parametras tinka bet kokia funkcija, kuri grąžina tekstą. Pvz.:
def reversed_text(text):
    return text[::-1]

In [8]:
print(upper_text("aukstesnio lygio funkcija", reversed_text))

AJICKNUF OIGYL OINSETSKUA


In [9]:
# Dabar pamėginsime analogišką rezultatą išgauti rašant dekoratorių. 
# Kaip matome, dekoratorius rašomas labai panašiai, kaip ir mūsų aukštesnio rango funkcija, tik jos turinys "suvyniojamas" į apvalkalą (wrapper). 
# Į dekoratoriaus parametrus dedame funkciją, kurią jis "apgaubs". Wrapper parametruose - tos funkcijos parametrai.
def upper_decorator(func):
    def wrapper(text):
        if type(text) != str:
            raise ValueError("Argument type must be a string.")
        some_text = func(text)
        return some_text.upper()
    return wrapper

In [12]:
# Parašius dekoratorių, python mums leidžia "apvilkti" savo funkcijas naudojant tokią sintaksę:
# dekoratorius @upper_decorator
@upper_decorator
def return_text(some_text):
    return some_text
@upper_decorator
def reversed_text(text):
    return text[::-1]

print(return_text("tiesiog didziosiom tekstas"))
print(reversed_text("apverstas tekstas"))

TIESIOG DIDZIOSIOM TEKSTAS
SATSKET SATSREVPA


In [13]:
# Parašykime dekoratorių, kuris priima visas tris funkcijas:
# dekoratorius:
def lyginis_nelyginis(func):
    # sita funkcija visada vadinsis wrapper
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        if result % 2 != 0: #nelygus nuliui
            return result, "nelyginis"
        return result, "lyginis"
    return wrapper

In [15]:
# prieš tai nagrinėtas dekoratorius (grąžinantis ALL CAPS) turėjo trūkumą - jis priėmė funkcijas tik su fiksuotu kiekiu argumentų(1). 
# Galima rašyti lankstesnius dekoratorius. Tarkime, turime funkcijas, grąžinančias integer reikšmes:
@lyginis_nelyginis
def duok_desim():
    return 10

@lyginis_nelyginis
def daugyba(x, y):
    return x * y

@lyginis_nelyginis
def sumuoti_viskas(*args):
    return sum(args)


In [16]:
print(duok_desim())
print(daugyba(11, 7))
print(sumuoti_viskas(5, 4, 3, 2, 7))

(10, 'lyginis')
(77, 'nelyginis')
(21, 'nelyginis')


In [40]:
# pats dekoratorius neturi savyje metaduomenu
def lyginis_nelyginis(func):
    def wrapper(*args, **kwargs):
        """nurodo ar funkcijos rezultatas yra lyginis/nelyginis skaicius"""
        result = func(*args, **kwargs)
        if result % 2 != 0: 
            return result, "nelyginis"
        return result, "lyginis"
    return wrapper

In [41]:
@lyginis_nelyginis
def duok_desim():
    """grazina 10"""
    return 10

In [42]:
# pabandę atsispausdinti metaduomenis, gautumėm:
print(duok_desim.__name__)
print(duok_desim.__doc__)

wrapper
nurodo ar funkcijos rezultatas yra lyginis/nelyginis skaicius


In [33]:
# bet jei panaudojam wraps dekoratoriu tada is dekoratoriaus nepaims to .__doc__
from functools import wraps
def lyginis_nelyginis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """nurodo ar funkcijos rezultatas yra lyginis/nelyginis skaicius"""
        result = func(*args, **kwargs)
        if result % 2 != 0: 
            return result, "nelyginis"
        return result, "lyginis"
    return wrapper

In [36]:
@lyginis_nelyginis
def duok_desim():
    """grazina 10"""
    return 10

In [37]:
# pabandę atsispausdinti metaduomenis, gautumėm:
print(duok_desim.__name__)
print(duok_desim.__doc__)


duok_desim
grazina 10


Kaip sukurti dekoratorių su parametrais:

In [50]:
from time import sleep

def uzvelavimas(sekundes):
    def uzvelavimas(func):
        def wrapper(*args, **kwargs):
            sleep(sekundes)
            print(f"pamiegojom {sekundes} sekundziu ir tada:")
            return func(*args, **kwargs)
        return wrapper
    return uzvelavimas

In [53]:
def args_counter(func):
    def wrapper(*args, **kwargs):
        print(f"Argumentu: {len(args)}, raktiniu argumentu: {len(kwargs)}")
        return func(*args, **kwargs)
    return wrapper

In [51]:
# kiek kartu kartojam cikla:
@uzvelavimas(2)
def skaiciuojam_iki_argumento(kartai):
    for skaicius in range(kartai):
        print(skaicius+1)

skaiciuojam_iki_argumento(5)

pamiegojom 2 sekundziu ir tada:
1
2
3
4
5


In [54]:
@uzvelavimas(3)
@args_counter
def sumuojam(*args):
    return sum(args)

print(sumuojam(10, 20, 33, 11, -77, 21))

pamiegojom 3 sekundziu ir tada:
Argumentu: 6, raktiniu argumentu: 0
18


Dabar ant funkcijos sumavimas uždėjome du dekoratorius – uzvelavimas (jis užvėlina programos paleidimą 3 sekundėmis) ir args_counter (jis suskaičiuoja funkcijai paduotus argumentus). Paleidus šią funkciją, ji pereina ir per abu dekoratorius ir padaro atitinkamą veiksmą.